# सत्र 2 – न्यूनतम RAG पाइपलाइन

Foundry Local और sentence-transformers embeddings का उपयोग करके एक हल्की Retrieval-Augmented Generation पाइपलाइन बनाएं।


### व्याख्या: डिपेंडेंसी इंस्टॉलेशन
इस पाइपलाइन के लिए न्यूनतम पैकेज इंस्टॉल करता है:
- `foundry-local-sdk` स्थानीय मॉडल प्रबंधन के लिए (यदि केवल BASE_URL पथ का उपयोग नहीं कर रहे हैं)।
- `openai` संगत SDK संरचनाओं के लिए (कुछ उपयोगिताएँ)।
- `sentence-transformers` एम्बेडिंग के लिए।
- `numpy` वेक्टर गणित के लिए।
पुनः चलाना सुरक्षित है; यदि पर्यावरण पहले से संतुष्ट है तो इसे छोड़ दें।


# परिदृश्य
यह नोटबुक पूरी तरह से स्थानीय रूप से चलने वाली एक न्यूनतम Retrieval-Augmented Generation (RAG) पाइपलाइन बनाती है:
- Foundry Local मॉडल से जुड़ती है (SDK या BASE_URL के माध्यम से स्वचालित पहचान करती है)।
- एक छोटा इन-मेमोरी दस्तावेज़ संग्रह बनाती है और इसे Sentence Transformers के साथ एम्बेड करती है।
- पारदर्शिता के लिए साधारण वेक्टर समानता पुनर्प्राप्ति (कोई बाहरी इंडेक्स नहीं) लागू करती है।
- कई HTTP बैकअप मार्गों (`/v1/chat/completions`, `/v1/completions`, `/v1/responses`) के माध्यम से ग्राउंडेड जनरेशन अनुरोध जारी करती है।
- एक `answer()` हेल्पर प्रदान करती है जो प्रारंभिक प्रयास विफल होने पर वैकल्पिक मॉडल रूपों को पुनः प्रयास करती है।

इसका उपयोग बड़े संग्रह, स्थायी वेक्टर स्टोर्स, या मूल्यांकन मेट्रिक्स पर स्केल करने से पहले एक डायग्नोस्टिक टेम्पलेट के रूप में करें (RAG मूल्यांकन नोटबुक देखें)।


In [5]:
# Install dependencies
!pip install -q foundry-local-sdk openai sentence-transformers numpy

### व्याख्या: मुख्य आयात
मुख्य लाइब्रेरी लोड करता है जो एम्बेडिंग और स्थानीय अनुमान के लिए आवश्यक हैं:
- SentenceTransformer घनी वेक्टर एम्बेडिंग के लिए।
- FoundryLocalManager (वैकल्पिक) स्थानीय सेवा को प्रबंधित करने के लिए।
- OpenAI क्लाइंट परिचित ऑब्जेक्ट संरचनाओं के लिए (हालांकि बाद में हम सीधे HTTP का उपयोग करते हैं)।


In [6]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI

### व्याख्या: टॉय डॉक्यूमेंट कॉर्पस
डोमेन कथनों की एक छोटी इन-मेमोरी सूची को परिभाषित करता है। पुनर्प्राप्ति और ग्राउंडिंग जैसे पाइपलाइन यांत्रिकी पर ध्यान केंद्रित बनाए रखने के लिए पुनरावृत्ति को तेज और नियंत्रित रखता है, न कि डेटा प्रबंधन पर।


In [7]:
DOCS = [
    'Foundry Local provides an OpenAI-compatible local inference endpoint.',
    'Retrieval Augmented Generation improves answer grounding by injecting relevant context.',
    'Edge AI reduces latency and preserves privacy via local execution.',
    'Small Language Models can offer competitive quality with lower resource usage.',
    'Vector similarity search retrieves semantically relevant documents.'
]

### व्याख्या: कनेक्शन, मॉडल चयन और एम्बेडिंग प्रारंभ
मजबूत कनेक्शन लॉजिक:
1. वैकल्पिक रूप से स्पष्ट `BASE_URL` (शुद्ध HTTP पथ) का उपयोग करता है, अन्यथा FoundryLocalManager पर निर्भर करता है।
2. `/v1/models` की जांच करता है और सबसे उपयुक्त ठोस मॉडल आईडी का चयन करता है (सटीक उपनाम > मानक परिवार > पहला उपलब्ध)।
3. पुनः प्रयास लूप को लागू करता है, जिसे `FOUNDRY_CONNECT_RETRIES` और देरी के साथ कॉन्फ़िगर किया जा सकता है।
4. टॉय कॉर्पस के लिए SentenceTransformer एम्बेडिंग (सामान्यीकृत वेक्टर) प्रारंभ करता है।
5. पुनरुत्पादन क्षमता के लिए OpenAI SDK संस्करण को कैप्चर करता है।
यदि सेवा अनुपस्थित है, तो क्रैश होने के बजाय इसे शुरू करने के लिए मार्गदर्शन प्रिंट करता है।


In [12]:
import os, time, json, requests, re
# Native Foundry Local SDK preferred; fall back to explicit BASE_URL if provided
os.environ.setdefault('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
alias = os.getenv('FOUNDRY_LOCAL_ALIAS', os.getenv('TARGET_MODEL', 'phi-4-mini'))
base_url_env = os.getenv('BASE_URL', '').strip()
manager = None
client = None
endpoint = None

def _canonicalize(model_id: str) -> str:
    """Remove CUDA suffix and version tags from model name."""
    b = model_id.split(':')[0]
    return re.sub(r'-cuda.*', '', b)

try:
    if base_url_env:
        # Allow user override; normalize by removing trailing / and optional /v1
        root = base_url_env.rstrip('/')
        if root.endswith('/v1'):
            root = root[:-3]
        endpoint = root
        print(f'[INFO] Using explicit BASE_URL override: {endpoint}')
    else:
        from foundry_local import FoundryLocalManager
        manager = FoundryLocalManager(alias)
        # Manager endpoint already includes /v1 - remove it for our base
        raw_endpoint = manager.endpoint.rstrip('/')
        if raw_endpoint.endswith('/v1'):
            endpoint = raw_endpoint[:-3]
        else:
            endpoint = raw_endpoint
        print(f'[OK] Foundry Local manager endpoint: {manager.endpoint} | base={endpoint} | alias={alias}')
    
    # Probe models list (endpoint does NOT include /v1 here)
    models_resp = requests.get(endpoint + '/v1/models', timeout=5)
    models_resp.raise_for_status()
    payload = models_resp.json() if models_resp.headers.get('content-type','').startswith('application/json') else {}
    data = payload.get('data', []) if isinstance(payload, dict) else []
    ids = [m.get('id') for m in data if isinstance(m, dict)]
    
    # Select best matching model
    chosen = None
    if alias in ids:
        chosen = alias
    else:
        for mid in ids:
            if _canonicalize(mid) == _canonicalize(alias):
                chosen = mid
                break
    if not chosen and ids:
        chosen = ids[0]
    model_name = chosen or alias
    
    # Initialize OpenAI client
    from openai import OpenAI as _OpenAI
    client = _OpenAI(
        base_url=endpoint + '/v1',  # OpenAI client needs full base URL with /v1
        api_key=(getattr(manager, 'api_key', None) or os.getenv('API_KEY') or 'not-needed')
    )
    print(f'[OK] Model resolved: {model_name} (total_models={len(ids)})')
except Exception as e:
    print('[ERROR] Failed to initialize Foundry Local client:', e)
    client = None
    model_name = alias

# Expose BASE for downstream compatibility (without /v1)
BASE = endpoint

# Embeddings setup
embed_model_name = os.getenv('EMBED_MODEL', 'sentence-transformers/all-MiniLM-L6-v2')
try:
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embed_model_name)
    doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)
    print(f'[OK] Embedded {len(DOCS)} docs using {embed_model_name} shape={doc_emb.shape}')
except Exception as e:
    print('[ERROR] Embedding init failed:', e)
    embedder = None
    doc_emb = None

try:
    import openai as _openai
    openai_version = getattr(_openai, '__version__', 'unknown')
    print('OpenAI SDK version:', openai_version)
except Exception:
    openai_version = 'unknown'

if client is None:
    print('\nNEXT: Start/verify service then re-run this cell:')
    print('  foundry service start')
    print('  foundry model run phi-4-mini')
    print('  (optional) set BASE_URL=http://127.0.0.1:57127')

[OK] Foundry Local manager endpoint: http://127.0.0.1:59778/v1 | base=http://127.0.0.1:59778 | alias=phi-4-mini
[OK] Model resolved: deepseek-r1-distill-qwen-7b-cuda-gpu:0 (total_models=11)
[OK] Embedded 5 docs using sentence-transformers/all-MiniLM-L6-v2 shape=(5, 384)
OpenAI SDK version: 1.109.1


### व्याख्या: Retrieve फ़ंक्शन (वेक्टर समानता)
`retrieve(query, k=3)` क्वेरी को एन्कोड करता है, कोसाइन समानता (सामान्यीकृत वेक्टर पर डॉट प्रोडक्ट) की गणना करता है, और शीर्ष-k डॉक इंडेक्स लौटाता है। यह पारदर्शिता के लिए न्यूनतम और इन-मेमोरी रहता है।


In [9]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return sims.argsort()[::-1][:k]

### व्याख्या: SDK-आधारित जनरेशन और उत्तर सहायक
Foundry Local SDK और OpenAI-संगत क्लाइंट विधियों का उपयोग करके पुनः निर्मित, मैनुअल कच्चे HTTP पोस्ट के बजाय:
- प्राथमिक पथ: `client.chat.completions.create` (संरचित संदेश)।
- बैकअप: `client.completions.create` (पुरानी प्रॉम्प्ट) और फिर `client.responses.create` (सरलीकृत प्रतिक्रियाओं API)।
- वैकल्पिक मॉडल आईडी (RAW बनाम स्ट्रिप्ड ALT) को सामान्य करता है ताकि संगतता बढ़ाई जा सके।
- `answer()` शीर्ष-k प्राप्त दस्तावेज़ों से एक आधारित प्रॉम्प्ट बनाता है और क्रमबद्ध प्रयास ट्रेस रिकॉर्ड करता है।
यह तर्क को पठनीय बनाए रखता है, जबकि OpenAI-संगत एंडपॉइंट्स के विकास के साथ-साथ सहज गिरावट की पेशकश करता है।


In [14]:
# SDK-based generation (Foundry Local manager + OpenAI client methods)
import re, time, json

def _strip_model_name(name: str) -> str:
    """Strip CUDA suffix and version tags from model name."""
    base = name.split(':')[0]
    base = re.sub(r'-cuda.*', '', base)
    return base

# Use the actual resolved model name from connection cell
RAW_MODEL = model_name
ALT_MODEL = _strip_model_name(RAW_MODEL)

def _try_via_client(messages, prompt, model_id: str, max_tokens=220, temperature=0.2):
    """Try generating response using OpenAI client with multiple fallback routes."""
    attempts = []
    
    # 1. Try chat.completions endpoint (preferred for chat models)
    try:
        resp = client.chat.completions.create(
            model=model_id, 
            messages=messages, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        content = resp.choices[0].message.content
        attempts.append(('chat.completions', 200, (content or '')[:160]))
        if content and content.strip():
            return content, attempts
    except Exception as e:
        attempts.append(('chat.completions', None, str(e)[:160]))
    
    # 2. Try legacy completions endpoint
    try:
        comp = client.completions.create(
            model=model_id, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        txt = comp.choices[0].text if comp.choices else ''
        attempts.append(('completions', 200, (txt or '')[:160]))
        if txt and txt.strip():
            return txt, attempts
    except Exception as e:
        attempts.append(('completions', None, str(e)[:160]))
    
    return None, attempts

def retrieve(query, k=3):
    """Retrieve top-k most similar documents using cosine similarity."""
    if embedder is None or doc_emb is None:
        raise RuntimeError("Embeddings not initialized.")
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    scores = doc_emb @ q_emb
    idxs = np.argsort(scores)[::-1][:k]
    return idxs

def answer(query, k=3, max_tokens=220, temperature=0.2, try_alternate=True):
    """
    Answer a query using RAG pipeline:
    1. Retrieve relevant documents using vector similarity
    2. Generate grounded response using Foundry Local model via OpenAI SDK
    
    Args:
        query: User question
        k: Number of documents to retrieve
        max_tokens: Maximum tokens for generation
        temperature: Sampling temperature
        try_alternate: Whether to try alternate model name on failure
    
    Returns:
        Dictionary with query, answer, docs, context, route, and tried attempts
    """
    if client is None:
        raise RuntimeError('Model client not initialized. Re-run connection cell after starting Foundry Local.')
    if embedder is None or doc_emb is None:
        raise RuntimeError('Embeddings not initialized.')
    
    # Retrieve relevant documents
    idxs = retrieve(query, k=k)
    context = '\n'.join(f'Doc {i}: {DOCS[i]}' for i in idxs)
    
    # Construct grounded generation prompt
    system_content = 'Use ONLY provided context. If insufficient, say "I\'m not sure."'
    user_content = f'Context:\n{context}\n\nQuestion: {query}'
    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content}
    ]
    prompt = f'System: {system_content}\n{user_content}\nAnswer:'
    
    # Try generation with primary model
    tried = []
    ans, attempts = _try_via_client(messages, prompt, RAW_MODEL, max_tokens=max_tokens, temperature=temperature)
    tried.append({'model': RAW_MODEL, 'attempts': attempts})
    
    if ans and ans.strip():
        return {
            'query': query, 
            'answer': ans.strip(), 
            'docs': idxs.tolist(), 
            'context': context, 
            'route': 'chat-first', 
            'tried': tried
        }
    
    # Try alternate model name if available
    if try_alternate and ALT_MODEL != RAW_MODEL:
        ans2, attempts2 = _try_via_client(messages, prompt, ALT_MODEL, max_tokens=max_tokens, temperature=temperature)
        tried.append({'model': ALT_MODEL, 'attempts': attempts2})
        if ans2 and ans2.strip():
            return {
                'query': query, 
                'answer': ans2.strip(), 
                'docs': idxs.tolist(), 
                'context': context, 
                'route': 'chat-alt', 
                'tried': tried
            }
    
    # All routes failed
    return {
        'query': query, 
        'answer': 'I\'m not sure. (All SDK routes failed)', 
        'docs': idxs.tolist(), 
        'context': context, 
        'route': 'failed', 
        'tried': tried
    }

print('[INFO] SDK generation mode active.')
print(f'       RAW_MODEL = {RAW_MODEL}')
print(f'       ALT_MODEL = {ALT_MODEL}')

[INFO] SDK generation mode active.
       RAW_MODEL = deepseek-r1-distill-qwen-7b-cuda-gpu:0
       ALT_MODEL = deepseek-r1-distill-qwen-7b


In [15]:
# Self-test cell: validates connectivity, embeddings, and answer() basic functionality (SDK mode)
import math, pprint

def rag_self_test(sample_query: str = 'Why use RAG with local inference?', expect_docs: int = 3):
    report = {'base': BASE, 'raw_model': RAW_MODEL, 'alt_model': ALT_MODEL}
    if not BASE:
        report['error'] = 'BASE not resolved'
        return report
    if embedder is None or doc_emb is None:
        report['error'] = 'Embeddings not initialized'
        return report
    if getattr(doc_emb, 'shape', (0,))[0] != len(DOCS):
        report['warning_embeddings'] = f"doc_emb count {getattr(doc_emb,'shape',('?'))} mismatch DOCS {len(DOCS)}"
    try:
        idxs = retrieve(sample_query, k=expect_docs)
        report['retrieved_indices'] = idxs.tolist() if hasattr(idxs, 'tolist') else list(idxs)
    except Exception as e:
        report['error_retrieve'] = str(e)
        return report
    try:
        ans = answer(sample_query, k=expect_docs, max_tokens=80, temperature=0.2)
        report['route'] = ans.get('route')
        report['answer_preview'] = ans.get('answer','')[:160]
        if ans.get('route') == 'failed':
            report['warning_generation'] = 'All SDK routes failed for sample query'
    except Exception as e:
        report['error_generation'] = str(e)
    return report

pprint.pprint(rag_self_test())

{'alt_model': 'deepseek-r1-distill-qwen-7b',
 'answer_preview': 'Okay, so I need to figure out why someone would use '
                   'Retrieval Augmented Generation (RAG) with local inference. '
                   'Let me start by understanding each part of the qu',
 'base': 'http://127.0.0.1:59778',
 'raw_model': 'deepseek-r1-distill-qwen-7b-cuda-gpu:0',
 'retrieved_indices': [0, 3, 1],
 'route': 'chat-first'}


### व्याख्या: बैच क्वेरी स्मोक टेस्ट
कई प्रतिनिधि उपयोगकर्ता प्रश्नों को `answer()` के माध्यम से निष्पादित करता है ताकि यह सुनिश्चित किया जा सके:
- पुनर्प्राप्ति सूचकांक संभावित सहायक दस्तावेज़ों से मेल खाते हैं।
- फॉलबैक रूटिंग काम करती है (रूट मान 'failed' नहीं है)।
- उत्तर ग्राउंडिंग निर्देश का पालन करते हैं (कोई कल्पना नहीं)।
अंतिम परिणाम ऑब्जेक्ट को तदर्थ निरीक्षण के लिए कैप्चर करता है।


In [16]:
# Quick test queries

queries = [

    "Why use RAG with local inference?",

    "What does vector similarity search do?",

    "Explain privacy benefits."

]



last_result = None

for q in queries:

    try:

        r = answer(q)

        last_result = r

        print(f"Q: {q}\nA: {r['answer']}\nDocs: {r['docs']}\n---")

    except Exception as e:

        print(f"Failed answering '{q}': {e}")



last_result

Q: Why use RAG with local inference?
A: Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.

First, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.

Then, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.

Now, combining RAG with local inference. The context s

{'query': 'Explain privacy benefits.',
 'answer': 'Okay, so I need to explain the privacy benefits mentioned in the provided context. Let me look at the context again. The context includes three documents:\n\nDoc 2 says Edge AI reduces latency and preserves privacy via local execution.\nDoc 3 mentions Small Language Models can offer competitive quality with lower resource usage.\nDoc 1 states Retrieval Augmented Generation improves answer grounding by injecting relevant context.\n\nThe question is about explaining the privacy benefits. So, I should focus on the parts of the context that talk about privacy. \n\nLooking at Doc 2, it mentions Edge AI reduces latency and preserves privacy via local execution. That seems directly related to privacy. I think "local execution" means that the AI processes data on the device itself rather than sending it to a server. This could mean that data doesn\'t have to be transmitted, which might help protect user privacy because it avoids centralizing d

### व्याख्या: एकल उत्तर सुविधा कॉल
आसान कॉपी/पेस्ट पुन: उपयोग या आगे संदर्भ के लिए अंतिम त्वरित एकल-प्रश्न कॉल। पहले वार्म-अप क्वेरी के बाद `answer()` के इडेम्पोटेंट उपयोग को प्रदर्शित करता है।


In [17]:
result = answer('Why use RAG with local inference?')
result

{'query': 'Why use RAG with local inference?',
 'answer': "Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.\n\nFirst, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.\n\nThen, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.\n\nNow, combining RAG with local


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
